# Section: Encrypted Deep Learning


# Encrypted Computations in PySyft

In [23]:
!pip install syft

In [0]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0715 01:27:16.022779 140376042014592 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0715 01:27:16.041859 140376042014592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
# we assigned one of the parties, secure_worker, to generate the random numbers 
# we have trust that the secure_worker, one of the parties, does not know Alice or Bob (they also do not know each other)
# use the parameter crypto_provider to assign on of the parties

x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker) # notice the number of the parties where the secrete is shared

In [0]:
bob._objects

We have shared two secretes with three parties. Let's try some computations remotely on these secretes:

In [0]:
# adding secretes on remote machines
z = x + y
z.get() # returns and decodes

In [0]:
# subtracting secretes on remote machines

z = x - y
z.get()

NameError: ignored

In [0]:
# multiplying secretes on remote machines

z = x * y
z.get()

NameError: ignored

In [0]:
# boolean operations on remote machines
z = x > y
z.get()

NameError: ignored

In [0]:
z = x < y
z.get()

NameError: ignored

In [0]:
z = x == y
z.get()

NameError: ignored

In [0]:
# double check that your implementation is using the fix_precision() function when dealing with float values
# reverse this function using the float_precision() function

x = th.tensor([1.2, 2.2, 3.2, 4.2])
y = th.tensor([2.1, -1.1, 1.1, 0.1])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

NameError: ignored

In [0]:
z = x + y
z.get().float_precision()

NameError: ignored

In [0]:
z = x - y
z.get().float_precision()

tensor([-0.9000,  3.3000,  2.1000,  4.1000])

In [0]:
z = x * y
z.get().float_precision()

tensor([ 2.5200, -2.4200,  3.5200,  0.4200])

In [0]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [0]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [0]:
z = x == x
z.get().float_precision()

tensor([1., 1., 1., 1.])

# Lesson: Encrypted Deep Learning in PyTorch

### Build your algorithms and Model

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x



def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        


In [0]:
model = Net()

#train the model
train()

tensor(1.2584)
tensor(1.7743)
tensor(6.3854)
tensor(9.6886)
tensor(4.9202)
tensor(0.8759)
tensor(0.7188)
tensor(0.6353)
tensor(0.5494)
tensor(0.4642)
tensor(0.3769)
tensor(0.2793)
tensor(0.2002)
tensor(0.1391)
tensor(0.0896)
tensor(0.0641)
tensor(0.0427)
tensor(0.0313)
tensor(0.0226)
tensor(0.0161)


In [0]:
# run predictions
model(data)

tensor([[ 0.0470],
        [-0.0036],
        [ 1.0402],
        [ 0.9093]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)
encrypted_model

Net(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)

In [0]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:14775828503 -> alice:14984268194]
 	-> (Wrapper)>[PointerTensor | me:10647128836 -> bob:16718500379]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:64953866464 -> alice:12993828591]
 	-> (Wrapper)>[PointerTensor | me:44406466286 -> bob:42492479853]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:5612843384 -> alice:99643612496]
 	-> (Wrapper)>[PointerTensor | me:10966840289 -> bob:36578350869]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:77714413229 -> alice:86677182728]
 	-> (Wrapper)>[PointerTensor | me:13802423202 -> bob:504642660

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0480],
        [-0.0030],
        [ 1.0380],
        [ 0.9080]])

# Reuse the MNIST NN from the previous classes (firs week of classes) to train the classifier with a Secure Federated learning appraoch.

In [1]:
import numpy
import syft as sy
import torch as th
from torch import nn, optim
from torchvision import datasets, transforms

hook = sy.TorchHook(th)

W0715 04:08:36.339025 139721948288896 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0715 04:08:36.355575 139721948288896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# Create workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

train_set = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)

# Get dataset and pass to workers
federated_train_loader = sy.FederatedDataLoader(train_set.federate((bob, alice)), 
                                                 batch_size=64, shuffle=True)

In [0]:
# Create model
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()


In [0]:
# Train distributed models
def train(epochs=1):  
  # Send model to workers
  model.fix_precision().share(alice, bob, crypto_provider=secure_worker, requires_grad=True)
  
  # Set up optimizer
  opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()
  
  for i in range(epochs):
    for images, labels in federated_train_loader:
      images.resize_(64, 784)
            
      # Forward pass
      opt.zero_grad()
      
      output = model(images)
      loss = criterion(output, labels)
      loss.backward()

      opt.step()
      
    # Print model update
    model.get().float_precision()
    print('Weights: ' + str(model.weight.data))
    print('Bias:' + str(model.bias.data))
#     print('Loss ' + str(loss.data))
    

In [0]:
train()